In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np
import tensorflow as tf

In [4]:
housing = datasets.fetch_california_housing()

In [5]:
m, n = housing.data.shape

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [10]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [46]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size) 
    X_batch = scaled_housing_data_plus_bias[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices] 
    return X_batch, y_batch

In [47]:
n_epochs = 1000
learning_rate = 0.1

In [48]:
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed = 42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize (mse)
init = tf.global_variables_initializer()

In [49]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [51]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [52]:
with tf.Session() as sess:                                                     
    sess.run(init)                                                                

    for epoch in range(n_epochs):                                                 
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()  
        

In [54]:
file_writer.close()

In [55]:
# run Tensorboard using the following command from the virualenv prompt of your Tensorflow insallation
# 'tensorboard --logdir tf_logs/'
# it will show the URL where the tensorboard is running
# 'TensorBoard 1.12.0 at http://moshiur-ubuntu:6006 (Press CTRL+C to quit)'
# click on the URL or copy-paste it in your favorite web browser, you will see the Tensorboad running
# showing MSE in the'SCALARS' tab and the computation graph in the 'GRAPHS' tab.